In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install the wheel
!pip install /kaggle/input/models/youneseloiarm/damas-library-001/other/default/1/damas-package/dist/damas-0.1.0-py3-none-any.whl

# Test the import
from damas import *

# If your script has functions, you can test them
# For example, if your script has a function named 'process_data':
# result = damas.process_data(...)

print("✓ damas successfully imported as a library!")

Processing /kaggle/input/models/youneseloiarm/damas-library-001/other/default/1/damas-package/dist/damas-0.1.0-py3-none-any.whl
✓ damas successfully imported as a library!


In [3]:
!pip install cairosvg imageio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00


In [4]:
import sys
from pathlib import Path
from IPython.display import SVG, display, clear_output, HTML
import os
import json
import time
import pandas as pd
from pathlib import Path
import random
import cairosvg
import imageio
import io
import re
from damas import *

In [5]:
class DamasSVGRenderer:
    def __init__(self, square_size=60):
        self.s = square_size
        self.board_px = 8 * self.s

    def render(self, game, title=None):
        s = self.s
        svg = []

        height = self.board_px + (30 if title else 0)
        svg.append(
            f'<svg width="{self.board_px}" height="{height}" '
            f'xmlns="http://www.w3.org/2000/svg">'
        )

        if title:
            svg.append(
                f'<text x="{self.board_px//2}" y="20" '
                f'text-anchor="middle" font-size="16">{title}</text>'
            )
            y_offset = 30
        else:
            y_offset = 0

        # Board
        for r in range(8):
            for c in range(8):
                x = c * s
                y = r * s + y_offset

                # Light / dark squares
                dark = (r + c) % 2 == 1
                color = "#B58863" if dark else "#F0D9B5"

                svg.append(
                    f'<rect x="{x}" y="{y}" width="{s}" height="{s}" fill="{color}"/>'
                )

                if not dark:
                    continue

                piece = game.board.get_piece(r, c)
                if piece:
                    self._draw_piece(svg, piece, x, y, s)

        svg.append("</svg>")
        return SVG("".join(svg))

    def _draw_piece(self, svg, piece, x, y, s):
        cx, cy = x + s/2, y + s/2
        r = s * 0.35

        fill = "#C40003" if piece.color == RED else "#0033A0"

        svg.append(
            f'<circle cx="{cx}" cy="{cy}" r="{r}" '
            f'fill="{fill}" stroke="black" stroke-width="2"/>'
        )

        if piece.king:
            svg.append(
                f'<text x="{cx}" y="{cy+6}" '
                f'text-anchor="middle" font-size="{s*0.45}" fill="gold">♛</text>'
            )



In [6]:
def play_local_svg(p1, p2, delay=0.6, max_moves=200):
    game = DamasGame()
    players = {RED: p1, BLUE: p2}
    renderer = DamasSVGRenderer()
    move_count = 0

    clear_output(wait=True)
    display(renderer.render(game, title="Spanish Damas — Start"))
    time.sleep(delay)

    while not game.is_game_over() and move_count < max_moves:
        current = game.get_current_player()
        player = players[current]

        state = game.get_state_representation()
        board_only = game.render_board_only()
        print("state :",state)
        print("board_only :",board_only)
        action = player.prompt(state, schema=DamasMove)

        ok = game.make_move(action)
        move_count += 1

        clear_output(wait=True)
        display(
            renderer.render(
                game,
                title=f"Move {move_count} — {current} ({player.name})"
            )
        )
        time.sleep(delay)

        if not ok:
            game.winner = BLUE if current == RED else RED
            break

    clear_output(wait=True)
    display(renderer.render(game, title="🏁 GAME OVER"))
    print("Winner:", game.winner)
def play_local_svg(
    p1,
    p2,
    delay=0.6,
    max_moves=200,
    record=True,
    gif_path="damas_game.gif",
    fps=2
):
    """
    Play Spanish Damas with:
    - Board | Game State layout
    - Optional GIF recording
    """

    game = DamasGame()
    players = {RED: p1, BLUE: p2}
    renderer = DamasSVGRenderer()
    move_count = 0

    frames = []  # store images

    # ---------- convert SVG to PNG frame ----------
    def capture_frame(svg_obj):
        if not record:
            return

        svg_str = svg_obj.data if hasattr(svg_obj, "data") else str(svg_obj)

        png_bytes = cairosvg.svg2png(bytestring=svg_str.encode("utf-8"))
        image = imageio.v3.imread(io.BytesIO(png_bytes))
        frames.append(image)

    # ---------- layout renderer ----------
    def show_layout(game, title, state=None):
        svg_board = renderer.render(game, title=title)

        state_content = f"""
        <div style="font-family: monospace; white-space: pre-wrap; padding:15px;">
            <h3>Game State</h3>
            {state if state else "Start"}
        </div>
        """

        layout = f"""
        <div style="display:flex; gap:20px; align-items:flex-start;">
            <div>
                {svg_board.data if hasattr(svg_board, "data") else svg_board}
            </div>
            <div style="min-width:300px; border-left:1px solid #ddd;">
                {state_content}
            </div>
        </div>
        """

        clear_output(wait=True)
        display(HTML(layout))

        capture_frame(svg_board)  # record frame

    # ---------- START ----------
    show_layout(game, "Spanish Damas — Start")
    time.sleep(delay)

    # ---------- GAME LOOP ----------
    while not game.is_game_over() and move_count < max_moves:
        current = game.get_current_player()
        player = players[current]

        state = game.get_state_representation()
        action = player.prompt(state, schema=DamasMove)

        ok = game.make_move(action)
        move_count += 1

        show_layout(
            game,
            f"Move {move_count} — {current} ({player.name})",
            state=state
        )
        time.sleep(delay)

        if not ok:
            game.winner = BLUE if current == RED else RED
            break

    # ---------- GAME OVER ----------
    show_layout(game, "🏁 GAME OVER")
    print("Winner:", game.winner)

    # ---------- SAVE GIF ----------
    if record and frames:
        imageio.mimsave(gif_path, frames, fps=fps)
        print(f"\n✅ GIF saved → {gif_path}")


In [7]:
if __name__ == "__main__":
        play_local_svg(RandomBot(), GreedyBot(), delay=0.5,record=True)

Winner: R

✅ GIF saved → damas_game.gif


In [8]:
if __name__ == "__main__":
        play_local_svg(AdvancedDamasBot(), GreedyBot(), delay=0.5,record=True)

Winner: R

✅ GIF saved → damas_game.gif
